# 🔒 Federated Learning: Privacy-Preserving ML

**Author**: Data Science Master System  
**Difficulty**: ⭐⭐⭐⭐⭐ Expert  
**Time**: 90 minutes  
**Prerequisites**: Deep Learning, Distributed Systems

## Learning Objectives
- Understand federated learning fundamentals
- Implement FedAvg algorithm
- Apply differential privacy
- Use Flower framework for production FL

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from typing import List, Dict, Tuple
import copy

np.random.seed(42)
torch.manual_seed(42)

## 1. Federated Learning Concepts

In [ ]:
concepts = '''
🔒 FEDERATED LEARNING

Traditional ML:     Data → Central Server → Train → Model
Federated ML:       Model → Devices → Local Train → Aggregate

KEY PRINCIPLES:
1. Data never leaves the device
2. Only model updates are shared
3. Central server aggregates updates
4. Privacy is preserved by design

USE CASES:
- Healthcare: Train on patient data across hospitals (HIPAA compliant)
- Finance: Fraud detection without sharing transaction data
- Mobile: Keyboard prediction without uploading messages
- IoT: Edge devices with sensitive sensor data
'''
print(concepts)

## 2. Simple Model for FL

In [ ]:
class SimpleNN(nn.Module):
    """Simple neural network for demonstration."""
    def __init__(self, input_dim=10, hidden_dim=32, output_dim=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
    
    def forward(self, x):
        return self.net(x)

model = SimpleNN()
print(f"Model parameters: {sum(p.numel() for p in model.parameters())}")

## 3. FedAvg Algorithm

In [ ]:
class FederatedAveraging:
    """Federated Averaging (FedAvg) implementation."""
    
    def __init__(self, global_model: nn.Module, num_clients: int):
        self.global_model = global_model
        self.num_clients = num_clients
    
    def client_update(self, model: nn.Module, data: torch.Tensor, 
                      labels: torch.Tensor, epochs: int = 5, lr: float = 0.01):
        """Train model on client's local data."""
        optimizer = torch.optim.SGD(model.parameters(), lr=lr)
        criterion = nn.CrossEntropyLoss()
        
        model.train()
        for _ in range(epochs):
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        return model.state_dict()
    
    def aggregate(self, client_weights: List[Dict], sample_counts: List[int]):
        """Weighted average of client model weights."""
        total_samples = sum(sample_counts)
        
        # Initialize aggregated weights
        aggregated = copy.deepcopy(client_weights[0])
        for key in aggregated:
            aggregated[key] = aggregated[key] * 0  # Zero out
        
        # Weighted sum
        for weights, count in zip(client_weights, sample_counts):
            weight = count / total_samples
            for key in aggregated:
                aggregated[key] += weights[key] * weight
        
        return aggregated
    
    def train_round(self, client_data: List[Tuple]):
        """Execute one round of federated training."""
        client_weights = []
        sample_counts = []
        
        for data, labels in client_data:
            # Each client gets a copy of global model
            client_model = copy.deepcopy(self.global_model)
            
            # Local training
            weights = self.client_update(client_model, data, labels)
            client_weights.append(weights)
            sample_counts.append(len(data))
        
        # Aggregate
        new_weights = self.aggregate(client_weights, sample_counts)
        self.global_model.load_state_dict(new_weights)
        
        return self.global_model

# Demonstration
fedavg = FederatedAveraging(SimpleNN(), num_clients=5)
print("✅ FedAvg initialized")

## 4. Differential Privacy

In [ ]:
class DifferentialPrivacy:
    """Add differential privacy to gradients."""
    
    def __init__(self, epsilon: float = 1.0, delta: float = 1e-5, max_grad_norm: float = 1.0):
        self.epsilon = epsilon
        self.delta = delta
        self.max_grad_norm = max_grad_norm
    
    def clip_gradients(self, model: nn.Module):
        """Clip gradients to max_grad_norm."""
        total_norm = 0
        for p in model.parameters():
            if p.grad is not None:
                total_norm += p.grad.data.norm(2).item() ** 2
        total_norm = total_norm ** 0.5
        
        clip_coef = self.max_grad_norm / (total_norm + 1e-6)
        if clip_coef < 1:
            for p in model.parameters():
                if p.grad is not None:
                    p.grad.data.mul_(clip_coef)
    
    def add_noise(self, model: nn.Module, noise_scale: float):
        """Add Gaussian noise to gradients."""
        for p in model.parameters():
            if p.grad is not None:
                noise = torch.randn_like(p.grad) * noise_scale
                p.grad.data.add_(noise)

dp = DifferentialPrivacy(epsilon=1.0)
print(f"DP configured: ε={dp.epsilon}, δ={dp.delta}")

## 5. Flower Framework

In [ ]:
flower_example = '''
# Flower: Production Federated Learning Framework

# Server (server.py)
import flwr as fl

strategy = fl.server.strategy.FedAvg(
    min_fit_clients=2,
    min_evaluate_clients=2,
    min_available_clients=2,
)

fl.server.start_server(
    server_address="0.0.0.0:8080",
    config=fl.server.ServerConfig(num_rounds=3),
    strategy=strategy,
)

# Client (client.py)
import flwr as fl

class MyClient(fl.client.NumPyClient):
    def get_parameters(self, config):
        return [p.numpy() for p in model.parameters()]
    
    def fit(self, parameters, config):
        # Train locally
        train_model(model, trainloader)
        return self.get_parameters({}), len(trainloader), {}
    
    def evaluate(self, parameters, config):
        loss, accuracy = test_model(model, testloader)
        return float(loss), len(testloader), {"accuracy": accuracy}

fl.client.start_numpy_client(server_address="127.0.0.1:8080", client=MyClient())
'''
print(flower_example)

## 6. Comparison & Trade-offs

In [ ]:
import pandas as pd

comparison = pd.DataFrame({
    'Aspect': ['Privacy', 'Data Transfer', 'Accuracy', 'Communication', 'Complexity'],
    'Centralized': ['Low', 'High (all data)', 'Best', 'Low', 'Low'],
    'Federated': ['High', 'Low (only weights)', '~5% lower', 'High (rounds)', 'Medium'],
    'FL + DP': ['Very High', 'Low + noise', '~10% lower', 'High', 'High']
})

print("📊 ML Paradigm Comparison:")
display(comparison)

## 🎯 Key Takeaways
1. FL keeps data local - privacy by design
2. FedAvg: simple but effective aggregation
3. Differential privacy adds noise for extra protection
4. Use Flower for production deployments
5. Trade-off: privacy vs accuracy (~5-10% loss)

**Next**: 27_quantum_ml.ipynb